In [ ]:
%matplotlib inline
import numpy as np
import azureml.core
from azureml.core import Workspace
from azureml.core.dataset import Dataset
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import glob2
import pandas as pd

### Select the  model to be evaluated from workspace

In [ ]:
workspace = Workspace.from_config()
workspace
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

checkboxes = []

for experiment_name, experiment in workspace.experiments.items():
    checkbox = widgets.Checkbox(
        value=False,
        description=experiment_name
    )
    display(checkbox)
    checkboxes.append(checkbox)

### Download the models on your local system for evaluation

In [ ]:
import shutil
# Get the selected experiments.
selected_experiments = []
for checkbox in checkboxes:
    if checkbox.value == True:
        selected_experiments.append(checkbox.description)

# Get folder.
temp_path = "logs"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

# For each selected experiment download logs of all completed runs.
for selected_experiment in selected_experiments:
    print("Experiment: {}".format(selected_experiment))
    experiment = workspace.experiments.get(selected_experiment)
    for run_index, run in enumerate(list(experiment.get_runs())[::-1]):
        log_path = os.path.join(temp_path, experiment.name, "run_{:02d}".format(run_index + 1))
#         if run.status == "Failed" and run.id == 'q3-depthmap-plaincnn-height_1595403832_d4528115':
        if  run.id == 'q3-depthmap-plaincnn-height-95k_1597988908_42c4ef33':
            print("Run: {}".format(run_index + 1))
            run.download_files(output_directory='logs/q3-depthmap-plaincnn-height-95k/run_03/', output_paths=None, batch_size=100, append_prefix=False)
#             run.download_files(prefix=".h5", output_directory=log_path, output_paths=None, batch_size=100, append_prefix=False)

print("Done.")

### Load the models

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
model = load_model('evaluation_95k_30082020/q3-depthmap-plaincnn-height-100-95k/run_03/outputs/best_model.h5')
# summarize model.
model.summary()

## preprocessing the inputs 

In [ ]:
image_target_height = 240
image_target_width =180

def tf_load_pickle(path):
    def py_load_pickle(path):
        depthmap, targets = pickle.load(open(path, "rb"))
        depthmap = preprocess_depthmap(depthmap)
        depthmap = depthmap/depthmap.max()
        depthmap = tf.image.resize(depthmap, (image_target_height, image_target_width))
        targets = preprocess_targets(targets, targets_indices)
        return depthmap, targets
    
    depthmap, targets = tf.py_function(py_load_pickle, [path], [tf.float32, tf.float32])
    depthmap.set_shape((image_target_height, image_target_width, 1))
    targets.set_shape((len(targets_indices,)))
    return depthmap, targets

In [ ]:
def preprocess_targets(targets, targets_indices):
    if targets_indices is not None:
        targets = targets[targets_indices]
    return targets.astype("float32")

def preprocess_depthmap(depthmap):
    # TODO here be more code.
    return depthmap.astype("float32")

In [ ]:
targets_indices = [0] # height =0 , weight =1

### Show a sample from the testset

In [ ]:
paths = '../testdepthmap1/1585551618-hlby208u8z/pc_1585551618-hlby208u8z_1593156356859_100_000.p'
depthmap, targets = pickle.load(open(paths, "rb"))
depthmap = preprocess_depthmap(depthmap)
depthmap = depthmap/depthmap.max()
print("depthmap_max:",depthmap.max())
depthmap = tf.image.resize(depthmap, (image_target_height, image_target_width))
targets = preprocess_targets(targets, targets_indices)
depthmap.set_shape((image_target_height, image_target_width, 1))
# targets.set_shape((len(targets_indices,)))
plt.imshow(np.squeeze(depthmap))

### Preprocess the samples from testset 

In [ ]:
def preprocess(path):
    depthmap, targets = pickle.load(open(path, "rb"))
    depthmap = preprocess_depthmap(depthmap)
#     depthmap = depthmap/depthmap.max()
    depthmap = depthmap/7.5
    depthmap = tf.image.resize(depthmap, (image_target_height, image_target_width))
    targets = preprocess_targets(targets, targets_indices)
    depthmap.set_shape((image_target_height, image_target_width, 1))
    return depthmap,targets

### Perform the prediction

In [ ]:
from tqdm.notebook import tqdm
predictions =[]
prediction_folder = glob2.glob('/mnt/depthmap/depthmap_testset/scans/*/*/') ## mount the dataset and provide the absolute path

In [ ]:
predictions = []
for qrcode in tqdm(prediction_folder):
    depthmaps_pred = []
    labels = []
    depthfiles = []
    depthmaps = glob2.glob(qrcode+'*.p')
    for files in depthmaps:
        depths,targets = preprocess(files)
        depthmaps_pred.append(depths)
        labels.append(targets)
        depthfiles.append(files)
    files_to_predict  =tf.stack(depthmaps_pred)
    inference = model.predict(files_to_predict)
    predictions.append([qrcode,depthfiles,np.squeeze(inference),labels])

In [ ]:
## putting the predictions in a dataframe
df = pd.DataFrame([])
for i in tqdm(range(len(predictions))):
    label = np.array(predictions[i][3])
    label =label.flatten()
    data =pd.DataFrame({'qrcode':predictions[i][0],'artifacts': predictions[i][1],'predicted':predictions[i][2],'GT':label})
    df = df.append(data)
df.head()

In [ ]:
## function to extract the qrcode from the artifacts 
def qrcode(row):
    qrc = row['artifacts'].split('/')[5]
    return qrc

In [ ]:
df['qrcode'] = df.apply(qrcode,axis=1)
df.head()

In [ ]:
df['artifacts'].iloc[1] # sample of how the artifacts path looks like for me, modify it accordingly to suit your path dependency

In [ ]:
len(df['qrcode'].unique()) ## total number of scans

In [ ]:
## function to extract the scantype of artifacts and display the number 
def scantype(row):
    scans = row['artifacts'].split('/')[6]
    return scans

In [ ]:
"## keys for different scan type
"- 100 - standing front scan\"
""- 101 - standing 360 scan\",
"\"- 102 - standing back scan\",\n",
"\"- 200 - lying front scan\",\n",
"\"- 201 - lying side scan\",\n",
"\"- 202 - lying back scan\"\n"

In [ ]:
df['scantype'] = df.apply(scantype, axis=1)
df['scantype'].value_counts()

## group the results of artifacts by qrcode and scantype by taking mean across the same scantype

In [ ]:
MAE = df.groupby(['qrcode','scantype']).mean()
MAE

## error between predicted and ground truth

In [ ]:
def avgerror(row):
    difference =row['GT'] - row['predicted']
    return difference

In [ ]:
MAE['error'] = MAE.apply(avgerror, axis=1)
MAE

In [ ]:
## error margin on various ranges
evaluation_accuracies = [.2,.4,.8,1.2,2,2.5,3,4,5,6]

In [ ]:
## froming the unique name for the index values
model_name = 'q3-depthmap-plaincnn-height-100-95k'
run_no ='_front_run_03'
complete_name = model_name+run_no

## calculating accuracies across the scantypes

In [ ]:
Scan = '_360'
full_model_name = complete_name + Scan
rot = MAE.iloc[MAE.index.get_level_values('scantype') == '101']
accuracy_list =[]
for acc in evaluation_accuracies:
    predicted_rot = rot[(rot['error']<=acc) & (rot['error']>=-acc)]
    rot_accuracy = len(predicted_rot)/len(rot)*100
    print("total accuracy {} for 360:{}".format(acc,rot_accuracy))
    accuracy_list.append(rot_accuracy)
rotdata =pd.DataFrame(accuracy_list)
rotdata =rotdata.T
rotdata.columns = evaluation_accuracies
rotdata.rename(index={0:full_model_name}, inplace=True)
rotdata

In [ ]:
Scan = '_back'
full_model_name = complete_name + Scan
back = MAE.iloc[MAE.index.get_level_values('scantype') == '102']
print(len(back))
accuracy_list =[]
for acc in evaluation_accuracies:
    predicted_back = back[(back['error']<=acc) & (back['error']>=-acc)]
    back_accuracy = len(predicted_back)/len(back)*100
    print("total accuracy {} for back:{}".format(acc,back_accuracy))
    accuracy_list.append(back_accuracy)
backdata =pd.DataFrame(accuracy_list)
backdata =backdata.T
backdata.columns = evaluation_accuracies
backdata.rename(index={0:full_model_name}, inplace=True)
backdata

In [ ]:
Scan = '_front'
full_model_name = complete_name + Scan
front= MAE.iloc[MAE.index.get_level_values('scantype') == '100']
print("total front samples:",len(front))
accuracy_list =[]
for acc in evaluation_accuracies:
    predicted_front = front[(front['error']<=acc) & (front['error']>=-acc)]
    front_accuracy = len(predicted_front)/len(front)*100
    print("total accuracy {} for front:{}".format(acc,front_accuracy))
    accuracy_list.append(front_accuracy)
frontdata =pd.DataFrame(accuracy_list)
frontdata =frontdata.T
frontdata.columns = evaluation_accuracies
frontdata.rename(index={0:full_model_name}, inplace=True)
frontdata

In [ ]:
Scan = '_lyingfront'
full_model_name = complete_name + Scan
lyingfront= MAE.iloc[MAE.index.get_level_values('scantype') == '200']
print("total lyingfront samples:",len(lyingfront))
accuracy_list =[]
for acc in evaluation_accuracies:
    predicted_front = lyingfront[(lyingfront['error']<=acc) & (lyingfront['error']>=-acc)]
    front_accuracy = len(predicted_front)/len(lyingfront)*100
    print("total accuracy {} for lyingfront:{}".format(acc,front_accuracy))
    accuracy_list.append(front_accuracy)
lyingfrontdata =pd.DataFrame(accuracy_list)
lyingfrontdata =lyingfrontdata.T
lyingfrontdata.columns = evaluation_accuracies
lyingfrontdata.rename(index={0:full_model_name}, inplace=True)
lyingfrontdata

In [ ]:
Scan = '_lyingrot'
full_model_name = complete_name + Scan
lyingrot= MAE.iloc[MAE.index.get_level_values('scantype') == '201']
print("total lyingrot samples:",len(lyingrot))
accuracy_list =[]
for acc in evaluation_accuracies:
    predicted_front = lyingrot[(lyingrot['error']<=acc) & (lyingrot['error']>=-acc)]
    front_accuracy = len(predicted_front)/len(lyingrot)*100
    print("total accuracy {} for lyingfront:{}".format(acc,front_accuracy))
    accuracy_list.append(front_accuracy)
lyingrotdata =pd.DataFrame(accuracy_list)
lyingrotdata =lyingrotdata.T
lyingrotdata.columns = evaluation_accuracies
lyingrotdata.rename(index={0:full_model_name}, inplace=True)
lyingrotdata

In [ ]:
Scan = '_lyingback'
full_model_name = complete_name + Scan
lyingback= MAE.iloc[MAE.index.get_level_values('scantype') == '202']
print("total lyingback samples:",len(lyingback))
accuracy_list =[]
for acc in evaluation_accuracies:
    predicted_back = lyingback[(lyingback['error']<=acc) & (lyingback['error']>=-acc)]
    back_accuracy = len(predicted_back)/len(lyingback)*100
    print("total accuracy {} for lyingback:{}".format(acc,back_accuracy))
    accuracy_list.append(back_accuracy)
lyingbackdata =pd.DataFrame(accuracy_list)
lyingbackdata =lyingbackdata.T
lyingbackdata.columns = evaluation_accuracies
lyingbackdata.rename(index={0:full_model_name}, inplace=True)
lyingbackdata

## combining the results for all accuracies

In [ ]:
result  = pd.concat([frontdata, rotdata,backdata,lyingfrontdata,lyingrotdata,lyingbackdata])
result

In [ ]:
result.index.name = 'Model_Scantype'
# alldata.set_names('Model', level=None, inplace=True)
result

In [ ]:
## Saving the model results in csv file
result.to_csv('evaluation_95k_30082020/q3-depthmap-plaincnn-height-95k/run_03.csv',index=True)

In [ ]:
complete_results =pd.concat([run_03,run_04,front])
complete_results = complete_results.round(2) ## saving the results upto two decimal places

In [ ]:
complete_results.to_csv('Model_results/95k_data_v0/results.csv',index=True)